<a href="https://colab.research.google.com/github/huggingface/blog/blob/notebook_update_may15/notebooks/01_how_to_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background-color: pink;">
  Notebook written in collaboration with <a href="https://github.com/aditya-malte">Aditya Malte</a>.
  <br>
  The Notebook is on GitHub, so contributions are more than welcome.
</div>
<br>
<div style="background-color: yellow;">
  Aditya wrote another notebook with a slightly different use case and methodology, please check it out.
  <br>
  <a target="_blank" href="https://gist.github.com/aditya-malte/2d4f896f471be9c38eb4d723a710768b">
    https://gist.github.com/aditya-malte/2d4f896f471be9c38eb4d723a710768b
  </a>
</div>


# How to train a new language model from scratch using Transformers and Tokenizers

### Notebook edition (link to blogpost [link](https://huggingface.co/blog/how-to-train)). Last update May 15, 2020


Over the past few months, we made several improvements to our [`transformers`](https://github.com/huggingface/transformers) and [`tokenizers`](https://github.com/huggingface/tokenizers) libraries, with the goal of making it easier than ever to **train a new language model from scratch**.

In this post we’ll demo how to train a “small” model (84 M parameters = 6 layers, 768 hidden size, 12 attention heads) – that’s the same number of layers & heads as DistilBERT – on **Esperanto**. We’ll then fine-tune the model on a downstream task of part-of-speech tagging.


## 1. Find a dataset

First, let us find a corpus of text in Esperanto. Here we’ll use the Esperanto portion of the [OSCAR corpus](https://traces1.inria.fr/oscar/) from INRIA.
OSCAR is a huge multilingual corpus obtained by language classification and filtering of [Common Crawl](https://commoncrawl.org/) dumps of the Web.

<img src="https://huggingface.co/blog/assets/01_how-to-train/oscar.png" style="margin: auto; display: block; width: 260px;">

The Esperanto portion of the dataset is only 299M, so we’ll concatenate with the Esperanto sub-corpus of the [Leipzig Corpora Collection](https://wortschatz.uni-leipzig.de/en/download), which is comprised of text from diverse sources like news, literature, and wikipedia.

The final training corpus has a size of 3 GB, which is still small – for your model, you will get better results the more data you can get to pretrain on. 



In [1]:
# in this notebook we'll only get one of the files (the Oscar one) for the sake of simplicity and performance
!wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

--2020-09-01 22:48:27--  https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 2600:9000:21c7:da00:e:3ec7:c0:93a1, 2600:9000:21c7:4400:e:3ec7:c0:93a1, 2600:9000:21c7:a00:e:3ec7:c0:93a1, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|2600:9000:21c7:da00:e:3ec7:c0:93a1|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



## 2. Train a tokenizer

We choose to train a byte-level Byte-pair encoding tokenizer (the same as GPT-2), with the same special tokens as RoBERTa. Let’s arbitrarily pick its size to be 52,000.

We recommend training a byte-level BPE (rather than let’s say, a WordPiece tokenizer like BERT) because it will start building its vocabulary from an alphabet of single bytes, so all words will be decomposable into tokens (no more `<unk>` tokens!).


In [3]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
!pip install pytorch_block_sparse
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3oqil6ky
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-3oqil6ky
  Created wheel for transformers: filename=transformers-3.1.0-py3-none-any.whl size=884119 sha256=ceeaad6b01023b099368c10323439ad58b0aebe8d723dd3616e4119f48ff7175
  Stored in directory: /tmp/pip-ephem-wheel-cache-dz0ja5qp/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
tokenizers                         0.8.1rc2
transformers                       3.1.0


In [1]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 18.2 s, sys: 3.45 s, total: 21.7 s
Wall time: 3.69 s


Now let's save files to disk

In [20]:
!mkdir EsperBERTo
tokenizer.save_model("EsperBERTo")

mkdir: cannot create directory ‘EsperBERTo’: File exists


['EsperBERTo/vocab.json', 'EsperBERTo/merges.txt']

🔥🔥 Wow, that was fast! ⚡️🔥

We now have both a `vocab.json`, which is a list of the most frequent tokens ranked by frequency, and a `merges.txt` list of merges.

```json
{
	"<s>": 0,
	"<pad>": 1,
	"</s>": 2,
	"<unk>": 3,
	"<mask>": 4,
	"!": 5,
	"\"": 6,
	"#": 7,
	"$": 8,
	"%": 9,
	"&": 10,
	"'": 11,
	"(": 12,
	")": 13,
	# ...
}

# merges.txt
l a
Ġ k
o n
Ġ la
t a
Ġ e
Ġ d
Ġ p
# ...
```

What is great is that our tokenizer is optimized for Esperanto. Compared to a generic tokenizer trained for English, more native words are represented by a single, unsplit token. Diacritics, i.e. accented characters used in Esperanto – `ĉ`, `ĝ`, `ĥ`, `ĵ`, `ŝ`, and `ŭ` – are encoded natively. We also represent sequences in a more efficient manner. Here on this corpus, the average length of encoded sequences is ~30% smaller as when using the pretrained GPT-2 tokenizer.

Here’s  how you can use it in `tokenizers`, including handling the RoBERTa special tokens – of course, you’ll also be able to use it directly from `transformers`.


In [21]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./EsperBERTo/vocab.json",
    "./EsperBERTo/merges.txt",
)

In [22]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [23]:
tokenizer.encode("Mi estas Julien.")

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [24]:
tokenizer.encode("Mi estas Julien.").tokens

['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']

## 3. Train a language model from scratch

**Update:** This section follows along the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, using our new [`Trainer`](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py) directly. Feel free to pick the approach you like best.

> We’ll train a RoBERTa-like model, which is a BERT-like with a couple of changes (check the [documentation](https://huggingface.co/transformers/model_doc/roberta.html) for more details).

As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.


In [1]:
# Check that we have a GPU (you should have only one, as there is currently an issue with Pytorch and Long tensors for multi-gpu)
# If you have several GPUs, make sure to launch jupyter using  CUDA_VISIBLE_DEVICES=0 jupyter notebook .
!nvidia-smi

Wed Sep  2 09:55:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 27%   29C    P8    21W / 250W |     98MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:03:00.0 Off |                  N/A |
| 27%   27C    P8    19W / 250W |      1MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [1]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

### We'll define the following config for the model

In [2]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

Now let's re-create our tokenizer in transformers

In [3]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./EsperBERTo", max_len=512)

Finally let's initialize our model.

**Important:**

As we are training from scratch, we only initialize from a config, not from an existing pretrained model or checkpoint.

In [5]:
from pytorch_block_sparse import BlockSparseModelPatcher

from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config).cuda()
 
mp = BlockSparseModelPatcher()
mp.add_pattern("roberta\.encoder\.layer\.[0-9]+\.intermediate\.dense", {"density":0.25})
mp.add_pattern("roberta\.encoder\.layer\.[0-9]+\.output\.dense", {"density":0.25})
mp.patch_model(model)
    
# => 63 million parameters instead of 84 million parameters
model.num_parameters()


Patching 'roberta.encoder.layer.0.intermediate.dense' with density=0.25, in=768, out=3072,bias=True 
Patching 'roberta.encoder.layer.0.output.dense' with density=0.25, in=3072, out=768,bias=True 
Patching 'roberta.encoder.layer.1.intermediate.dense' with density=0.25, in=768, out=3072,bias=True 
Patching 'roberta.encoder.layer.1.output.dense' with density=0.25, in=3072, out=768,bias=True 
Patching 'roberta.encoder.layer.2.intermediate.dense' with density=0.25, in=768, out=3072,bias=True 
Patching 'roberta.encoder.layer.2.output.dense' with density=0.25, in=3072, out=768,bias=True 
Patching 'roberta.encoder.layer.3.intermediate.dense' with density=0.25, in=768, out=3072,bias=True 
Patching 'roberta.encoder.layer.3.output.dense' with density=0.25, in=3072, out=768,bias=True 
Patching 'roberta.encoder.layer.4.intermediate.dense' with density=0.25, in=768, out=3072,bias=True 
Patching 'roberta.encoder.layer.4.output.dense' with density=0.25, in=3072, out=768,bias=True 
Patching 'roberta.en

62861344

### Now let's build our training Dataset

We'll build our dataset by applying our tokenizer to our text file.

Here, as we only have one text file, we don't even need to customize our `Dataset`. We'll just use the `LineByLineDataset` out-of-the-box.

In [6]:
%%time

from transformers import LineByLineTextDataset

from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
import os

class LineByLineTextDataset2(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(self, tokenizer: PreTrainedTokenizer, file_path: str, block_size: int):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
#        logger.info("Creating features from dataset file at %s", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        batch_encoding = tokenizer(lines, add_special_tokens=True, truncation=True, max_length=block_size)
        self.examples = batch_encoding["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        ret = torch.tensor(self.examples[i], dtype=torch.long)
#        print(ret.shape)
        return ret

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./oscar.eo.txt",
    block_size=128,
)

CPU times: user 3min 48s, sys: 7.51 s, total: 3min 56s
Wall time: 45.3 s


Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Finally, we are all set to initialize our Trainer

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=16, # Adapt it to your size
    save_steps=10_000
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/home/lagunas/ml/anaconda3/lib/python3.7/site-packages/transformers/trainer.py:247: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,


### Start training

In [ ]:
%%time
trainer.train()

{'loss': 8.2496845703125, 'learning_rate': 4.9897389591200135e-05, 'epoch': 0.008208832703989493, 'step': 500}
{'loss': 7.70870703125, 'learning_rate': 4.979477918240027e-05, 'epoch': 0.016417665407978985, 'step': 1000}
{'loss': 7.5345732421875, 'learning_rate': 4.969216877360039e-05, 'epoch': 0.02462649811196848, 'step': 1500}
{'loss': 7.410927734375, 'learning_rate': 4.9589558364800525e-05, 'epoch': 0.03283533081595797, 'step': 2000}
{'loss': 7.316962890625, 'learning_rate': 4.948694795600066e-05, 'epoch': 0.04104416351994746, 'step': 2500}
{'loss': 7.22975, 'learning_rate': 4.9384337547200796e-05, 'epoch': 0.04925299622393696, 'step': 3000}
{'loss': 7.160671875, 'learning_rate': 4.928172713840092e-05, 'epoch': 0.05746182892792645, 'step': 3500}
{'loss': 7.11912109375, 'learning_rate': 4.9179116729601054e-05, 'epoch': 0.06567066163191594, 'step': 4000}
{'loss': 7.0350625, 'learning_rate': 4.9076506320801186e-05, 'epoch': 0.07387949433590543, 'step': 4500}
{'loss': 7.021046875, 'learn

/home/lagunas/ml/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 6.692453125, 'learning_rate': 4.784518141520276e-05, 'epoch': 0.17238548678377935, 'step': 10500}
{'loss': 6.67634375, 'learning_rate': 4.774257100640289e-05, 'epoch': 0.18059431948776883, 'step': 11000}
{'loss': 6.623, 'learning_rate': 4.7639960597603024e-05, 'epoch': 0.18880315219175833, 'step': 11500}
{'loss': 6.58925, 'learning_rate': 4.7537350188803156e-05, 'epoch': 0.19701198489574784, 'step': 12000}
{'loss': 6.548796875, 'learning_rate': 4.743473978000328e-05, 'epoch': 0.20522081759973732, 'step': 12500}
{'loss': 6.55353125, 'learning_rate': 4.7332129371203414e-05, 'epoch': 0.21342965030372682, 'step': 13000}
{'loss': 6.435515625, 'learning_rate': 4.7229518962403546e-05, 'epoch': 0.2216384830077163, 'step': 13500}
{'loss': 6.42075, 'learning_rate': 4.7126908553603685e-05, 'epoch': 0.2298473157117058, 'step': 14000}
{'loss': 6.338125, 'learning_rate': 4.702429814480381e-05, 'epoch': 0.23805614841569528, 'step': 14500}
{'loss': 6.2759375, 'learning_rate': 4.69216877360039

{'loss': 4.36825, 'learning_rate': 3.973895912001314e-05, 'epoch': 0.8208832703989493, 'step': 50000}
{'loss': 4.3549375, 'learning_rate': 3.9636348711213264e-05, 'epoch': 0.8290921031029388, 'step': 50500}
{'loss': 4.3388125, 'learning_rate': 3.9533738302413396e-05, 'epoch': 0.8373009358069282, 'step': 51000}
{'loss': 4.3333125, 'learning_rate': 3.943112789361353e-05, 'epoch': 0.8455097685109177, 'step': 51500}
{'loss': 4.3403125, 'learning_rate': 3.932851748481366e-05, 'epoch': 0.8537186012149073, 'step': 52000}
{'loss': 4.3163125, 'learning_rate': 3.922590707601379e-05, 'epoch': 0.8619274339188967, 'step': 52500}
{'loss': 4.2978125, 'learning_rate': 3.9123296667213925e-05, 'epoch': 0.8701362666228862, 'step': 53000}
{'loss': 4.305375, 'learning_rate': 3.902068625841406e-05, 'epoch': 0.8783450993268758, 'step': 53500}
{'loss': 4.3071875, 'learning_rate': 3.891807584961419e-05, 'epoch': 0.8865539320308652, 'step': 54000}
{'loss': 4.26525, 'learning_rate': 3.881546544081432e-05, 'epoch

{'loss': 4.1831875, 'learning_rate': 3.748153012641603e-05, 'epoch': 1.001477589886718, 'step': 61000}
{'loss': 4.1509375, 'learning_rate': 3.737891971761616e-05, 'epoch': 1.0096864225907076, 'step': 61500}
{'loss': 4.15725, 'learning_rate': 3.7276309308816285e-05, 'epoch': 1.0178952552946972, 'step': 62000}
{'loss': 4.1711875, 'learning_rate': 3.717369890001642e-05, 'epoch': 1.0261040879986867, 'step': 62500}
{'loss': 4.1484375, 'learning_rate': 3.707108849121655e-05, 'epoch': 1.034312920702676, 'step': 63000}
{'loss': 4.1004375, 'learning_rate': 3.696847808241668e-05, 'epoch': 1.0425217534066655, 'step': 63500}
{'loss': 4.11475, 'learning_rate': 3.686586767361681e-05, 'epoch': 1.050730586110655, 'step': 64000}
{'loss': 4.11825, 'learning_rate': 3.6763257264816946e-05, 'epoch': 1.0589394188146446, 'step': 64500}
{'loss': 4.094, 'learning_rate': 3.666064685601708e-05, 'epoch': 1.0671482515186341, 'step': 65000}
{'loss': 4.1365625, 'learning_rate': 3.655803644721721e-05, 'epoch': 1.0753

#### 🎉 Save final model (+ tokenizer + config) to disk

In [18]:
trainer.save_model("./EsperBERTo_sparse")

## 4. Check that the LM actually trained

Aside from looking at the training and eval losses going down, the easiest way to check whether our language model is learning anything interesting is via the `FillMaskPipeline`.

Pipelines are simple wrappers around tokenizers and models, and the 'fill-mask' one will let you input a sequence containing a masked token (here, `<mask>`) and return a list of the most probable filled sequences, with their probabilities.



In [11]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./EsperBERTo",
    tokenizer="./EsperBERTo"
)

Some weights of the model checkpoint at ./EsperBERTo were not used when initializing RobertaForMaskedLM: ['roberta.encoder.layer.0.attention.output.dense.weight_data', 'roberta.encoder.layer.0.attention.output.dense.full_weight', 'roberta.encoder.layer.0.intermediate.dense.weight_data', 'roberta.encoder.layer.0.intermediate.dense.full_weight', 'roberta.encoder.layer.0.output.dense.weight_data', 'roberta.encoder.layer.0.output.dense.full_weight', 'roberta.encoder.layer.1.attention.output.dense.weight_data', 'roberta.encoder.layer.1.attention.output.dense.full_weight', 'roberta.encoder.layer.1.intermediate.dense.weight_data', 'roberta.encoder.layer.1.intermediate.dense.full_weight', 'roberta.encoder.layer.1.output.dense.weight_data', 'roberta.encoder.layer.1.output.dense.full_weight', 'roberta.encoder.layer.2.attention.output.dense.weight_data', 'roberta.encoder.layer.2.attention.output.dense.full_weight', 'roberta.encoder.layer.2.intermediate.dense.weight_data', 'roberta.encoder.layer.2

In [12]:
# The sun <mask>.
# =>

fill_mask("La suno <mask>.")

[{'sequence': '<s>La suno sur.</s>',
  'score': 0.00045268586836755276,
  'token': 487,
  'token_str': 'Ġsur'},
 {'sequence': '<s>La suno tio.</s>',
  'score': 0.00044507990241982043,
  'token': 533,
  'token_str': 'Ġtio'},
 {'sequence': "<s>La suno'.</s>",
  'score': 0.00038873989251442254,
  'token': 11,
  'token_str': "'"},
 {'sequence': '<s>La suno Komunaĵo.</s>',
  'score': 0.0003828267799690366,
  'token': 891,
  'token_str': 'ĠKomunaÄµo'},
 {'sequence': '<s>La suno,.</s>',
  'score': 0.00037591546424664557,
  'token': 16,
  'token_str': ','}]

Ok, simple syntax/grammar works. Let’s try a slightly more interesting prompt:



In [37]:
fill_mask("Jen la komenco de bela <mask>.")

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.01814725436270237,
  'sequence': '<s> Jen la komenco de bela urbo.</s>',
  'token': 871},
 {'score': 0.015888698399066925,
  'sequence': '<s> Jen la komenco de bela vivo.</s>',
  'token': 1160},
 {'score': 0.015662025660276413,
  'sequence': '<s> Jen la komenco de bela tempo.</s>',
  'token': 1021},
 {'score': 0.015555007383227348,
  'sequence': '<s> Jen la komenco de bela mondo.</s>',
  'token': 945},
 {'score': 0.01412549614906311,
  'sequence': '<s> Jen la komenco de bela tago.</s>',
  'token': 1633}]

## 5. Share your model 🎉

Finally, when you have a nice model, please think about sharing it with the community:

- upload your model using the CLI: `transformers-cli upload`
- write a README.md model card and add it to the repository under `model_cards/`. Your model card should ideally include:
    - a model description,
    - training params (dataset, preprocessing, hyperparameters), 
    - evaluation results,
    - intended uses & limitations
    - whatever else is helpful! 🤓

### **TADA!**

➡️ Your model has a page on http://huggingface.co/models and everyone can load it using `AutoModel.from_pretrained("username/model_name")`.

[![tb](https://huggingface.co/blog/assets/01_how-to-train/model_page.png)](https://huggingface.co/julien-c/EsperBERTo-small)


If you want to take a look at models in different languages, check https://huggingface.co/models

[![all models](https://huggingface.co/front/thumbnails/models.png)](https://huggingface.co/models)
